In [8]:
import cv2

def is_tree(image, contour):
    x, y, w, h = cv2.boundingRect(contour)
    area = w * h

    # Expected area for a tree (approximately)
    expected_area = 7500

    # Calculate the ratio of the actual area to the expected area
    ratio = area / expected_area

    # If the ratio is above a certain threshold, consider it a tree
    threshold = 0.5
    if ratio > threshold:
        return True
    else:
        return False

def count_unique_trees(image, contours):
    tree_ids = set()
    for contour in contours:
        if is_tree(image, contour):
            # Use the contour area as a unique tree ID
            tree_id = cv2.contourArea(contour)
            tree_ids.add(tree_id)
    return len(tree_ids)

def draw_rounded_rectangle(image, contour, color):
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

video = cv2.VideoCapture('drone.mov')

#total_trees = 0
frame_count = 0

while True:
    ret, frame = video.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #unique_trees = count_unique_trees(frame, contours)
    #total_trees += unique_trees

    frame_count += 1
    cv2.putText(frame, f'Frame: {frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    for contour in contours:
        if is_tree(frame, contour):
            draw_rounded_rectangle(frame, contour, (0, 0, 255))

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

